<a href="https://colab.research.google.com/github/azoqi/Natural-Language-Processing/blob/main/ClassCoding3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem: we would like to implement a simple spell checker. We want to let our user type a sentence in English, and then print the most likely correct spelling of every word in the sentence.

Requirements: write a Python program that implements Min Edit Distance for string comparisons. You should accept one line of text typed on the terminal, tokenize that line of text to separate individual words, and then select the most likely correct spelling of each word in the sentence. You should use this list of 10,000 common English words to locate correct spellings: https://apiacoa.org/publications/teaching/datasets/google-10000-english.txtLinks to an external site.. Use Min Edit Distance to select the most similar word in the list for each word the user typed. Your program should print the most likely correct sentence.

In [1]:
import requests
import string

In [2]:
#Downloads the list of words from the URL.
#Assumes each line of the response is a word.
#Returns a list of lower-cased words.
def load_dictionary(url):
    response = requests.get(url)
    words = response.text.splitlines()
    return [word.strip().lower() for word in words if word.strip()]

In [3]:
#Compute the Levenshtein (minimum edit) distance between strings s and t.
def edit_distance(s, t):
    m, n = len(s), len(t)
    # Create a (m+1) x (n+1) matrix.
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the first column and first row.
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Fill in the matrix.
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s[i - 1] == t[j - 1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,      # deletion
                dp[i][j - 1] + 1,      # insertion
                dp[i - 1][j - 1] + cost  # substitution
            )
    return dp[m][n]


In [4]:
#Given a word and the dictionary (a list of words),
#return the dictionary word with the smallest edit distance.
#If there is a tie, the first encountered candidate is returned.
def correct_word(word, dictionary):
    # If the word is already in the dictionary, return it immediately.
    if word in dictionary:
        return word

    best_candidate = word
    best_distance = float('inf')
    for candidate in dictionary:
        dist = edit_distance(word, candidate)
        if dist < best_distance:
            best_distance = dist
            best_candidate = candidate
    return best_candidate

In [6]:
def main():
    dict_url = "https://apiacoa.org/publications/teaching/datasets/google-10000-english.txt"
    dictionary = load_dictionary(dict_url)
    dictionary_set = set(dictionary)

    sentence = input("Enter a sentence: ")
    tokens = sentence.split()
    corrected_tokens = []

    for token in tokens:
        stripped = token.strip(string.punctuation).lower()
        if not stripped:
            corrected_tokens.append(token)
            continue
        if stripped in dictionary_set:
            corrected = stripped
        else:
            corrected = correct_word(stripped, dictionary)
        corrected_tokens.append(corrected)

    # Reconstruct and print the corrected sentence.
    corrected_sentence = " ".join(corrected_tokens)
    print("\nMost likely correct sentence:")
    print(corrected_sentence)

if __name__ == "__main__":
    main()

Enter a sentence: hello worfd

Most likely correct sentence:
hello world
